In [1]:
import numpy as np
import pandas as pd

In [2]:
dtype = {'type': np.int8, 'clear_text': str,}
path = 'C:/Users/user/sentiment/clear_binary_twitts.csv' #home

In [3]:
df = pd.read_csv (path, sep=',', dtype=dtype)

In [4]:
def count_unic_words(data):
    
    number = 0
    dic = []
    i = 0
    for st in data:
        for w in str(st).split():
            if w not in dic:
                dic.append(w)
       
    number = len(dic)
    return number, dic

Этот ноутбук можно смотреть только после ноутбука (MyModel) ! Далее будет попытка улучшить (считерить) начальную версию модели.

# Схитрим? ~0.8 о_____О
В предыдущей модели мы записывали в словарь только слова из train выборки. В голову приходят мысли, что в реальной бизнес-задаче
у нас есть огромная база данных сообщений, в которой очень много слов. Что если мы будем иметь полную статистику о всех словах,
используемых в речи. Тогда мы будем знать, какие слова нейтральные, а какие используются чаще в позитивных/негативных сообщениях.
Чтобы имитировать такое "предобучение", попробуем обработать в словарь весь сет данных.

p.s. Предлагаю называть это имитацией предобученого словаря, а не допусканием дикого оверфита с:

In [5]:
n, l = count_unic_words(df['clear_text']) # n ~ 85к вместо 81к

In [6]:
def create_dic_words(n, l):
      
    nul1 = [int(0) for i in range(n)]
    nul2 = [int(0) for i in range(n)]    
    dic = {'word': l, 'pos': nul1, 'neg': nul2}

    return dic

In [7]:
dic = create_dic_words(n, l)

In [8]:
def filling_used_train_words(dataframe, dic):
    
    i = -1
    for index, row in dataframe.iterrows():
        i += 1
        
        spisok = row['clear_text']
        
        for w in str(spisok).split():
            if row['type'] == 1:
                dic.get('pos')[l.index(w)] += 1
            if row['type'] == -1:
                dic.get('neg')[l.index(w)] += 1

        if (i % 10000 == 0):
            print('обработана ', i, 'строка из 204150: ', i/2041.5, ' %\n')
    return dic

In [9]:
dictionary_train_words = filling_used_train_words(df, dic) # здесь будет сбитые %, так как 100% - это около 220к вместо 200

обработана  0 строка из 204150:  0.0  %

обработана  10000 строка из 204150:  4.898359049718344  %

обработана  20000 строка из 204150:  9.796718099436688  %

обработана  30000 строка из 204150:  14.695077149155033  %

обработана  40000 строка из 204150:  19.593436198873377  %

обработана  50000 строка из 204150:  24.49179524859172  %

обработана  60000 строка из 204150:  29.390154298310065  %

обработана  70000 строка из 204150:  34.28851334802841  %

обработана  80000 строка из 204150:  39.186872397746754  %

обработана  90000 строка из 204150:  44.0852314474651  %

обработана  100000 строка из 204150:  48.98359049718344  %

обработана  110000 строка из 204150:  53.88194954690179  %

обработана  120000 строка из 204150:  58.78030859662013  %

обработана  130000 строка из 204150:  63.678667646338475  %

обработана  140000 строка из 204150:  68.57702669605682  %

обработана  150000 строка из 204150:  73.47538574577517  %

обработана  160000 строка из 204150:  78.37374479549351  %

обра

In [10]:
nul3 = [int(0) for i in range(n)]
dictionary_train_words['value'] = nul3
for k in range(n):
    q = (dictionary_train_words['pos'][k] - dictionary_train_words['neg'][k])/(dictionary_train_words['pos'][k] + dictionary_train_words['neg'][k])
    dictionary_train_words['value'][k] = q

In [11]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.1, random_state=42)

In [12]:
def my_model_predict(data, dic):
    
    data = data.tolist()
    pred = [int(-1) for i in range(len(data))]
    
    for z in range(len(data)):
        p = 0
        for w in str(data[z]).split():
            if w in l:
                p += dic['value'][l.index(w)]
        if p >= 0:
            pred[z] = 1
    return pred

In [13]:
from sklearn.metrics import accuracy_score

In [14]:
print('My model ', accuracy_score(test['type'], my_model_predict(test['clear_text'], dictionary_train_words)))

My model  0.7965526362193617


Сразу приходят мысли применить предобученные вектора слов последних публикаций 2018 года. Такие как, ELMO и BERT.
Но, так как я ограничен размерами своей оперативной памятьи, попробовать это не могу.

Examples of resources with pretrained vectors:
http://docs.deeppavlov.ai/en/master/intro/pretrained_vectors.html
http://docs.deeppavlov.ai/en/master/components/classifiers.html

В первой ссылке есть вектора слов с твиттера - ELMo on Russian Twitter (8.5 Gb)